In [1]:
import sys
import numpy as np
import torch
import learn2learn as l2l
from torch import nn, optim
from torch.nn import functional as F
from torchvision import transforms

from PIL.Image import LANCZOS
from csv import writer

from PIL import Image
import pandas as pd

In [2]:
torch.cuda.is_available()

False

In [3]:
sys.path.append('/home/anuj/Desktop/Work/TU_Delft/research/implement/learning_to_meta-learn')
sys.path

['/home/anuj/Desktop/Work/TU_Delft/research/implement/learning_to_meta-learn/src',
 '/home/anuj/.vscode/extensions/ms-toolsai.jupyter-2022.6.1201981810/pythonFiles',
 '/home/anuj/.vscode/extensions/ms-toolsai.jupyter-2022.6.1201981810/pythonFiles/lib/python',
 '/home/anuj/anaconda3/envs/torch/lib/python39.zip',
 '/home/anuj/anaconda3/envs/torch/lib/python3.9',
 '/home/anuj/anaconda3/envs/torch/lib/python3.9/lib-dynload',
 '',
 '/home/anuj/anaconda3/envs/torch/lib/python3.9/site-packages',
 '/home/anuj/anaconda3/envs/torch/lib/python3.9/site-packages/IPython/extensions',
 '/home/anuj/.ipython',
 '/home/anuj/Desktop/Work/TU_Delft/research/implement/learning_to_meta-learn']

In [4]:
# from data.taskers import gen_tasks
from src.zoo.archs import CCVAE
# from src.zoo.trident_utils import setup, inner_adapt_trident, loss, accuracy
# from data.loaders import CUBirds200
from src.zoo.trident_utils import inner_adapt_trident, setup


In [5]:
device = 'cpu'
n_ways = 5
k_shots = 1
q_shots = 1
order = False
inner_lr = 0.001
meta_lr = 0.0001
reconst_loss = nn.MSELoss(reduction='none')
adapt_steps = 2
meta_batch_size = 10
download = False

In [6]:
class Args:
    def __init__(self, wt_ce, klwt, rec_wt, beta_l, beta_s):
        #args.wt_ce, args.klwt, args.rec_wt, args.beta_l, args.beta_s
        self.wt_ce = wt_ce
        self.klwt = klwt
        self.rec_wt = rec_wt
        self.beta_l = beta_l
        self.beta_s = beta_s
        self.dataset = 'miniimagenet'
        self.root = '../../dataset/'
        self.task_adapt = True
        self.task_adapt_fn = 'tafe'
        self.n_ways = 5
        self.k_shots = 1
        self.q_shots = 10
        self.zl = 64
        self.zs = 64
        self.reconstr = 'std'
        self.device = 'cpu'
        self.wn_channels = 32
        self.wm_channels = 64

args = Args(1, False, 1, 1, 1)

In [13]:
learner = torch.load('../logs/DELPO_tiered_5-way_1-shot_10-queries/exp5-tafe(wo relu)/model_59500.pt')

In [17]:
learner.args

Namespace(cnfg='/home/nfs/anujsingh/meta_lrng/files/temp/learning_to_meta-learn/configs/tiered-5,1/exp5-tafe.json', dataset='tiered', root='../../dataset', model_path='/home/nfs/anujsingh/meta_lrng/files/temp/learning_to_meta-learn/logs/DELPO_tiered_5-way_1-shot_15-queries/models-5-tafe', pretrained=None, n_ways=5, k_shots=1, q_shots=10, inner_adapt_steps_train=5, inner_adapt_steps_test=5, inner_lr=0.0015, meta_lr=0.00015, meta_batch_size=20, iterations=100000, wt_ce=150, klwt=False, rec_wt=0.01, beta_l=1, beta_s=1, task_adapt=True, task_adapt_fn='tafe', alpha=0.1, experiment='exp5-tafe', order=False, device='cuda', download=None, resume='No', iter_resume=0)

In [24]:
learner.modules

<bound method Module.modules of MAML(
  (module): CCVAE(
    (encoder): CEncoder(
      (net): Sequential(
        (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU()
        (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
        (4): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (5): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (6): ReLU()
        (7): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
        (8): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (9): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (10): ReLU()
        (11): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
        (12): Conv2d(32, 32, kernel_size=(3, 3), str

In [8]:
learner = CCVAE(in_channels=3, base_channels=32,
                        n_ways=args.n_ways, task_adapt=True, args=args, latent_dim_l=args.zl, latent_dim_s=args.zs)
opt = optim.Adam(learner.parameters(), 0.00015)

In [11]:
opt.load_state_dict(torch.load("./logs/DELPO_tiered_5-way_1-shot_10-queries/exp5-tafe(wo relu)/", map_location='cpu'))

IsADirectoryError: [Errno 21] Is a directory: '../logs/DELPO_tiered_5-way_1-shot_10-queries/exp5-tafe(wo relu)'

In [7]:
train_tasks, valid_tasks, _, learner = setup(
    args.dataset, args.root, args.n_ways, args.k_shots, args.q_shots, order, inner_lr, args.device, download=download, task_adapt=args.task_adapt, args=args)
opt = optim.Adam(learner.parameters(), meta_lr)
reconst_loss = nn.MSELoss(reduction='none')


In [8]:
ttask = train_tasks.sample()
model = learner.clone()

In [9]:
evaluation_loss, evaluation_accuracy = inner_adapt_trident(
    ttask, reconst_loss, model, args.n_ways, args.k_shots, args.q_shots, adapt_steps, args.device, False, args)

In [10]:
evaluation_loss

{'elbo': tensor(12.8777, grad_fn=<AddBackward0>),
 'label_kl': tensor(1.6585, grad_fn=<MeanBackward0>),
 'style_kl': tensor(9.3425, grad_fn=<MeanBackward0>),
 'reconstruction_loss': tensor(0.1450, grad_fn=<MeanBackward0>),
 'classification_loss': tensor(1.7316, grad_fn=<NllLossBackward0>)}

In [11]:
evaluation_loss['elbo'].backward()

In [12]:
torch.save(learner.state_dict(), './temp.pt')

In [9]:
learner1 = CCVAE(in_channels=3, base_channels=32, n_ways=args.n_ways, task_adapt=True, args=args, latent_dim_l=64, latent_dim_s=64)
learner1 = l2l.algorithms.MAML(learner1, first_order=order, lr=inner_lr, allow_nograd=False)
learner1.load_state_dict(torch.load('../logs/DELPO_miniimagenet_5-way_1-shot_10-queries/og/model_24000.pt'))

<All keys matched successfully>

In [10]:
learner1.state_dict()

OrderedDict([('module.encoder.net.0.weight',
              tensor([[[[nan, nan, nan],
                        [nan, nan, nan],
                        [nan, nan, nan]],
              
                       [[nan, nan, nan],
                        [nan, nan, nan],
                        [nan, nan, nan]],
              
                       [[nan, nan, nan],
                        [nan, nan, nan],
                        [nan, nan, nan]]],
              
              
                      [[[nan, nan, nan],
                        [nan, nan, nan],
                        [nan, nan, nan]],
              
                       [[nan, nan, nan],
                        [nan, nan, nan],
                        [nan, nan, nan]],
              
                       [[nan, nan, nan],
                        [nan, nan, nan],
                        [nan, nan, nan]]],
              
              
                      [[[nan, nan, nan],
                        [nan, nan, nan],
       

In [17]:
learner1.state_dict()['module.decoder.net.2.running_mean']

tensor([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
        nan, nan, nan, nan, nan, nan, nan, nan])

In [19]:
model = learner1.clone()
evaluation_loss, evaluation_accuracy = inner_adapt_trident(
    ttask, reconst_loss, model, args.n_ways, args.k_shots, args.q_shots, adapt_steps, args.device, False, args)

In [20]:
evaluation_loss

{'elbo': tensor(12.1259, grad_fn=<AddBackward0>),
 'label_kl': tensor(1.4813, grad_fn=<MeanBackward0>),
 'style_kl': tensor(8.8035, grad_fn=<MeanBackward0>),
 'reconstruction_loss': tensor(0.1288, grad_fn=<MeanBackward0>),
 'classification_loss': tensor(1.7124, grad_fn=<NllLossBackward0>)}